In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from sort import *

In [49]:
model = YOLO("yolov8m.pt")

In [50]:
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

In [51]:
line_video_1_in = [140, 480, 700, 310]
line_video_1_out = [80, 330, 620, 180]

line_video_2_in = [0, 225, 900, 225]
line_video_2_out = [0, 275, 900, 275]

In [52]:
counted_ids = set()
total_count = 0
in_ids = set()
out_ids = set()
cap = cv2.VideoCapture('people_video_2.mp4')
while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame , conf=0.5, classes=[0])
    
    detections = np.empty((0, 5))
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Convert tensors to CPU and Python types
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
            conf = float(box.conf[0].item())
            cls = int(box.cls[0].item())
            if cls == 0:
                current_array = np.array([x1, y1, x2, y2, conf], dtype=float)
                detections = np.vstack((detections, current_array))
    Results_tracker = tracker.update(detections)
    
    cv2.line(frame, (line_video_2_in[0], line_video_2_in[1]), (line_video_2_in[2], line_video_2_in[3]), (0, 0, 255), 3)
    cv2.line(frame, (line_video_2_out[0], line_video_2_out[1]), (line_video_2_out[2], line_video_2_out[3]), (0, 255, 255), 3)
    
    for result in Results_tracker:
        x1, y1, x2, y2, id = map(int, result)
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.circle(frame, (cx, cy) , 5, (0, 255, 0), cv2.FILLED)
        if line_video_2_in[0] < cx < line_video_2_in[2] and line_video_2_in[1] - 10 < cy < line_video_2_in[3] + 10 and id not in out_ids and id not in in_ids:
            out_ids.add(id)
            total_count += 1
        if line_video_2_out[0] < cx < line_video_2_out[2] and line_video_2_out[1] - 10 < cy < line_video_2_out[3] + 10 and id not in in_ids and id not in out_ids:
            out_ids.add(id)
            total_count -= 1
        cv2.putText(frame, f'ID: {id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    cv2.putText(frame, f'Total Count: {total_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # cv2.waitKey(0)
    
    cv2.imshow('People Counter', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 6.8ms
Speed: 1.0ms preprocess, 6.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6.4ms
Speed: 1.9ms preprocess, 6.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6.4ms
Speed: 1.2ms preprocess, 6.4ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6.3ms
Speed: 1.1ms preprocess, 6.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6.0ms
Speed: 1.2ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6.3ms
Speed: 0.9ms preprocess, 6.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6.2ms
Speed: 1.0ms preprocess, 6.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6.4ms
Speed: 0.9ms preprocess, 6.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384,

In [48]:
total_count

8